In [1]:
# We are using black as a code formatter
%load_ext nb_black

<IPython.core.display.Javascript object>

In [30]:
import requests
import json
import datetime
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import os

<IPython.core.display.Javascript object>

In [5]:
# pd.set_option('display.max_rows', None)
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
pd.set_option("display.max_colwidth", None)

<IPython.core.display.Javascript object>

## API Naciones Unidas

#### The base path for accessing the API is:
base_path ["https://population.un.org/dataportalapi/api/v1"](https://population.un.org/dataportalapi/api/v1)

#### Most common reported status codes
* 200 : Successful request
* 400 : Bad request
* 404 : Input parameters not found
* 406 : Requested output format not allowed
* 500 : Server error

#### Structure of API response (json)

* pageNumber : the current page of the response, which may have multiple pages
* pageSize : the number of records returned on the current page (a maximum of 100 records will be returned)
* previousPage : the path to the previous page of the response when multiple pages are returned
* nextPage : the path to the next page of the response when multiple pages are returned
* pages : the total number of pages in the response
* total : the total number of records in the response
* data : the actual data returned in the response

### Las diferentes fuentes

In [6]:
# Define target URL.
base_url = "https://population.un.org/dataportalapi/api/v1/sources"

# Call the API and convert the resquest into JSON object.
response = requests.get(base_url).json()

# Convert JSON object to data frame.
df = pd.json_normalize(response["data"])

print("df.shape", df.shape, end="\n")

# Display only relevant data.
df[["name", "sourceYear", "startYear", "endYear", "url"]].tail(3)

df.shape (26, 8)


,name,sourceYear,startYear,endYear,url
23,World Contraceptive Use 2022,2022,1950,2022,https://www.un.org/development/desa/pd/themes/family-planning
24,World Population Prospects,2022,1950,2100,https://population.un.org/wpp/
25,Estimates and Projections of Women of Reproductive Age Who Are Married or in a Union 2022,2022,1970,2030,https://www.un.org/development/desa/pd/content/fertility-and-marriage-0


<IPython.core.display.Javascript object>

### Los topicos

In [11]:
# Define target URL
base_url = "https://population.un.org/dataportalapi/api/v1/topics"

# Call the API and convert the resquest into JSON object
response = requests.get(base_url).json()

# Convert JSON object to data frame
df = pd.json_normalize(response["data"])

print("df.shape", df.shape, end="\n")

# Display only relevant data.
df[["name", "shortName"]]

df.shape (10, 4)


,name,shortName
0,Not applicable,NA
1,Population,Pop
2,Fertility,Fert
3,Mortality,Mort
4,International Migration,iMigration
5,Family Planning,FP
6,Marital Status,MarStat
7,All Components,All
8,Child Mortality,IGME
9,Maternal Mortality,MMEIG


<IPython.core.display.Javascript object>

### Los indicadores

In [6]:
# Define target URL.
base_url = "https://population.un.org/dataportalapi/api/v1/indicators"

# Call the API and convert the resquest into JSON object.
response = requests.get(base_url).json()

# Convert JSON object to data frame.
df = pd.json_normalize(response["data"])

print("df.shape", df.shape, end="\n")

# Display only relevant data.
df[["id", "name", "description", "shortName", "topicName", "topicShortName"]]

df.shape (60, 33)


,id,name,description,shortName,topicName,topicShortName
0,1,Contraceptive prevalence: Any method (Percent),Percentage of women of reproductive age (15-49 years) who are currently using any method of contraception,CPAnyP,Family Planning,FP
1,2,Contraceptive prevalence: Any modern method (Percent),Percentage of women of reproductive age (15-49 years) who are currently using any modern method of contraception,CPModP,Family Planning,FP
2,3,Contraceptive prevalence: Any traditional method (Percent),Percentage of women of reproductive age (15-49 years) who are currently using any traditional method of contraception,CPTrad,Family Planning,FP
3,4,Unmet need for family planning: Any method (Percent),Percentage of women of reproductive age (15-49 years) who want to stop or delay childbearing but are not using a method of contraception,UNMP,Family Planning,FP
4,5,Unmet need for family planning: Any modern method (Percent),Percentage of women of reproductive age (15-49 years) who want to stop or delay childbearing but are not using a modern method of contraception,UNMModP,Family Planning,FP
5,6,Total demand for family planning (Percent),Percentage of women of reproductive age (15-49 years) who are currently using any method of contraception or are having unmet need for family planning,DEMTot,Family Planning,FP
6,7,Demand for family planning satisfied by any method (Percent),Percentage of women of reproductive age (aged 15-49 years) who have their need for family planning satisfied with any methods,DEMAny,Family Planning,FP
7,8,Demand for family planning satisfied by any modern method (Percent),Percentage of women of reproductive age (aged 15-49 years) who have their need for family planning satisfied with modern methods,DEMMod,Family Planning,FP
8,9,Contraceptive prevalence: Any method (Number),Number of women of reproductive age (15-49 years) who are currently using any method of contraception,CPAnyN,Family Planning,FP
9,10,Contraceptive prevalence: Any modern method (Number),Number of women of reproductive age (15-49 years) who are currently using any modern method of contraception,CPModN,Family Planning,FP


<IPython.core.display.Javascript object>

### Areas geograficas

In [10]:
# Base url
base_url = "https://population.un.org/dataportalapi/api/v1"

# Creates the target URL, indicators, in this instance.
target = base_url + "/locations/"

# Get the response, which includes the first page of data as well as information on pagination and number of records.
response = requests.get(target)

# Converts call into JSON.
j = response.json()

# Converts JSON into a pandas DataFrame.
df = pd.json_normalize(
    j["data"]
)  # pd.json_normalize flattens the JSON to accomodate nested lists within the JSON structure.

# Loop until there are new pages with data.
while j["nextPage"] != None:
    # Reset the target to the next page.
    target = j["nextPage"]

    # call the API for the next page.
    response = requests.get(target)

    # Convert response to JSON format.
    j = response.json()

    # Store the next page in a data frame.
    df_temp = pd.json_normalize(j["data"])

    # Append next page to the data frame.
    df = pd.concat([df, df_temp], ignore_index=True)
df.sample(7)

,id,name,iso3,iso2,longitude,latitude
3,16,American Samoa,ASM,AS,-170.696182,-14.306021
137,508,Mozambique,MOZ,MZ,35.529564,-18.665695
62,233,Estonia,EST,EE,25.013607,58.595272
182,670,Saint Vincent and the Grenadines,VCT,VC,-61.191340,13.247750
12,48,Bahrain,BHR,BH,50.557701,26.066700
245,909,Oceania,OCE,OA,NaN,NaN
186,686,Senegal,SEN,SN,-14.452362,14.497401


<IPython.core.display.Javascript object>

### Areas geograficas con información del Banco Mundial

In [12]:
# Define target URL.
base_url = "https://population.un.org/dataportalapi/api/v1/locationsWithAggregates?pageNumber=1"

# Call the API and convert the resquest into JSON object.
response = requests.get(base_url).json()

# Convert JSON object to data frame.
df = pd.json_normalize(response)

# Get the response, which includes the first pages. Only 3.
pages = 3

# Converts call into JSON and concat to the previous data frame.
for page in range(2, pages + 1):
    # Reset the target to the next page
    target = f"https://population.un.org/dataportalapi/api/v1/locationsWithAggregates?pageNumber={page}"

    # Each iteration call the API and convert the resquest into JSON object.
    response = requests.get(target).json()

    # Each iteration convert JSON object to data frame.
    df_temp = pd.json_normalize(response)

    # Each iteration concat the data frames.
    df = pd.concat([df, df_temp], ignore_index=True)

print("df.shape", df.shape, end="\n")
df

# Display only relevant data. (Drop NaN, )
df_copy = (
    df[
        [
            "Id",
            "Name",
            "Iso2",
            "Iso3",
            "Longitude",
            "Latitude",
            "Region",
            "SubRegion",
            "WorldBankIncomeGroup",
            "UNDevelopmentGroup",
        ]
    ]
    .copy()
    .dropna()
)
# df_copy.to_parquet(f"../datasets/df_locationsWithAggregates.parquet")
df_copy.sample(7)

df.shape (278, 12)


,Id,Name,Iso2,Iso3,Longitude,Latitude,Region,SubRegion,WorldBankIncomeGroup,UNDevelopmentGroup
208,768,Togo,TG,TGO,0.824782,8.619543,Africa,Western Africa,Low-income countries,Least developed countries
11,44,Bahamas,BS,BHS,-77.396278,25.034281,Latin America and the Caribbean,Caribbean,High-income countries,Other developing regions
1,8,Albania,AL,ALB,20.168331,41.153332,Europe,Southern Europe,Upper-middle-income countries,Developed regions
93,356,India,IN,IND,78.962883,20.593683,Asia,Southern Asia,Lower-middle-income countries,Other developing regions
8,32,Argentina,AR,ARG,-63.616673,-38.416096,Latin America and the Caribbean,South America,Upper-middle-income countries,Other developing regions
43,174,Comoros,KM,COM,43.333302,-11.645500,Africa,Eastern Africa,Lower-middle-income countries,Least developed countries
127,474,Martinique,MQ,MTQ,-61.024420,14.631150,Latin America and the Caribbean,Caribbean,High-income countries,Other developing regions


<IPython.core.display.Javascript object>

### Convertimos los países escogidos en una lista de strings

In [13]:
# Stores indicator codes in a list
id_code = [str(code) for code in df_copy["Id"].values]

# Converts indicator code list into string to be used in later API call
id_code_string = ",".join(id_code)
id_code_string

'4,8,12,16,20,24,28,31,32,36,40,44,48,50,51,52,56,64,68,70,72,76,84,90,92,96,100,104,108,112,116,120,132,136,140,144,148,152,156,158,170,174,175,178,180,184,188,191,192,196,203,204,208,212,214,218,222,226,231,232,233,234,238,242,246,250,254,258,262,266,268,270,275,276,288,292,296,300,308,312,316,320,324,328,332,336,340,344,348,352,356,360,364,368,372,376,380,384,388,392,398,400,404,408,410,414,417,418,422,426,428,430,434,438,440,442,446,450,454,458,462,466,470,474,478,480,484,492,496,498,499,500,504,508,512,516,520,524,528,531,533,534,535,540,548,554,558,562,566,570,578,580,583,584,585,586,591,598,600,604,608,616,620,624,626,630,634,638,642,643,646,654,659,660,662,670,674,678,682,686,688,690,694,702,703,704,705,706,710,716,724,728,729,732,740,748,752,756,760,762,764,768,772,776,780,784,788,792,795,796,798,800,804,807,818,826,833,834,850,854,858,860,862,876,882,887,894'

<IPython.core.display.Javascript object>

* <mark>*Locations and Aggregate-locations differs in one country*</mark>
* <mark>*Aggregate-location has more features than Locations*</mark>

In [25]:
# the UN's API includes many entities which are not sovereign states,
# yet are listed as "Country". This workaround is a manuel fix for if
# one is only interested in countries by the classic definition
not_countries = [
    "American Samoa",
    "Bermuda",
    "British Virgin Islands",
    "Cayman Islands",
    "Mayotte",
    "Cook Islands",
    "Faroe Islands",
    "Falkland Islands (Malvinas)",
    "French Guiana",
    "French Polynesia",
    "Gibraltar",
    "Greenland",
    "Guadeloupe",
    "Guam",
    "China, Hong Kong SAR",
    "China, Macao SAR",
    "Martinique",
    "Montserrat",
    "Curaçao",
    "Aruba",
    "Sint Maarten (Dutch part)",
    "Bonaire, Sint Eustatius and Saba",
    "New Caledonia",
    "Niue",
    "Northern Mariana Islands",
    "Puerto Rico",
    "Réunion",
    "Saint Helena",
    "Anguilla",
    "Saint Pierre and Miquelon",
    "Tokelau",
    "Turks and Caicos Islands",
    "Isle of Man",
    "United States Virgin Islands",
    "Wallis and Futuna Islands",
]

<IPython.core.display.Javascript object>

La documentación nos brinda una función de ayuda para extraer infomación.

In [14]:
# Define a function that will take a relative path as an input, call the API, and return a dataframe
def callAPI(relative_path: str, topic_list: bool = False) -> pd.DataFrame:
    base_url = "https://population.un.org/dataportalapi/api/v1"
    target = (
        base_url + relative_path
    )  # Query string parameters may be appended here or directly in the provided relative path
    # Calls the API
    response = requests.get(target)
    # Reformats response into a JSON object
    j = response.json()
    # The block below will deal with paginated results.
    # If results not paginated, this will be skipped.
    try:
        # If results are paginated, they are transformed into a python dictionary.
        # The data may be accessed using the 'data' key of the dictionary.
        df = pd.json_normalize(j["data"])
        # As long as the nextPage key of the dictionary contains an address for the next API call, the function will continue to call the API and append the results to the dataframe.
        while j["nextPage"] is not None:
            response = requests.get(j["nextPage"])
            j = response.json()
            df_temp = pd.json_normalize(j["data"])
            df = pd.concat([df, df_temp], ignore_index=True)
    except:
        if topic_list:
            df = pd.json_normalize(j)
        else:
            df = pd.DataFrame(j)
    return df

<IPython.core.display.Javascript object>

### Vamos a usar la función para encontrar todos los indicadores de Población

In [17]:
# Uses callAPI function to get a list of Family Planning indicators
df_pop_indicators = callAPI("/topics/Pop/indicators", topic_list=False)
df_pop_indicators[
    ["indicatorId", "indicatorDescription", "sourceStartYear", "sourceEndYear"]
]

,indicatorId,indicatorDescription,sourceStartYear,sourceEndYear
0,53,The crude rate of natural change is the ratio of the natural change during the year (live births minus deaths) to the average population in that year. The value is expressed per 1 000 persons.,1950,2100
1,41,Female population of reproductive age (15-49 years),1950,2100
2,67,"Age that divides the population in two parts of equal size, that is, there are as many persons with ages above the median as there are with ages below the median. It is expressed as years.",1950,2100
3,52,"The difference between the number of live births and the number of deaths during the year. A positive natural change, also known as natural increase, occurs when live births outnumber deaths. A negative natural change, also named as natural decrease, occurs when live births are less numerous than deaths.",1950,2100
4,71,"Percentage of Total Population by various functional combination of age groups (0-14, 0-17, primary and secondary school ages, 15-24, 15-49, ..., 18+, 50+, etc.). De facto population as of 1 July of the year indicated. Figures are expressed per 100 population.",1950,2100
5,47,"Annual population by single age and by sex (interpolated data based on 5-year age groups and 5-year periods). De facto population as of 1 July of the year indicated classified by single age (0, 1, 2,.., 99, 100+).",1950,2100
6,46,"Annual population by five-year age groups and by sex (interpolated data based on 5-year periods). De facto population as of 1 July of the year indicated classified by five-year age groups (0-4, 5-9, 10-14,.., 95-99, 100+).",1950,2100
7,70,"De facto population as of 1 July of the year indicated classified by sex (male, female, both sexes combined) and by various functional combination of age groups (0-14, 0-17, primary and secondary school ages, 15-24, 15-49, ..., 18+, 50+, etc.). Data are presented in thousands.",1950,2100
8,50,Difference between the population sizes on 1 January of two consecutive years.,1950,2100
9,54,Number of persons per square Kilometer.,1950,2100


<IPython.core.display.Javascript object>

In [18]:
# Stores indicator codes in a list
indicator_pop_codes = [str(code) for code in df_pop_indicators["indicatorId"].values]

# Converts indicator code list into string to be used in later API call
indicator_pop_string = ",".join(indicator_pop_codes)
indicator_pop_string

'53,41,67,52,71,47,46,70,50,54,51,72,49'

<IPython.core.display.Javascript object>

### Vamos a usar la función para encontrar todos los indicadores de Mortalidad

In [19]:
# Uses callAPI function to get a list of Family Planning indicators
df_mort_indicators = callAPI("/topics/Mort/indicators", topic_list=False)
df_mort_indicators[
    ["indicatorId", "indicatorDescription", "sourceStartYear", "sourceEndYear"]
]

,indicatorId,indicatorDescription,sourceStartYear,sourceEndYear
0,79,"Central death rate for the age interval (x, x+n) where x is the initial age and n is the length of the interval. It is obtained as the ratio of the number of deaths by the number of person-years of exposure of the same age group (for a specified time period).",1950,2100
1,80,Central death rate between ages x and x + 1. It is obtained as the ratio of the number of deaths by the number of person-years of exposure of the same age interval (for a specified time period).,1950,2100
2,59,Number of deaths over a given period divided by the person-years lived by the population over that period.,1950,2100
3,69,Number of deaths by single age and by sex over a given period.,1950,2100
4,64,Number of deaths by age groups and by sex over a given period.,1950,2100
5,61,The average number of years of life expected by a hypothetical cohort of individuals who would be subject throughout their lives to the age-specific mortality rates of a given period.,1950,2100
6,75,"Expectation of life at age x is the average number of years remaining to be lived by those surviving to that age, based on a given set of age-specific rates of dying. It is derived by dividing the total person-years that would be lived beyond age x by the number of persons who survived to that age interval (Tx / lx).",1950,2100
7,76,"Expectation of life at age x is the average number of years remaining to be lived by those surviving to that age, based on a given set of age-specific rates of dying. It is derived by dividing the total person-years that would be lived beyond age x by the number of persons who survived to that age interval (Tx / lx).",1950,2100
8,62,Probability of dying between the fifteen and fiftieth birthdays.,1950,2100
9,63,Probability of dying between the fifteen and sixtieth birthdays.,1950,2100


<IPython.core.display.Javascript object>

In [20]:
# Stores indicator codes in a list
indicator_mort_codes = [str(code) for code in df_mort_indicators["indicatorId"].values]

# Converts indicator code list into string to be used in later API call
indicator_mort_string = ",".join(indicator_mort_codes)
indicator_mort_string

'79,80,59,69,64,61,75,76,62,63,81,82,77,78,60'

<IPython.core.display.Javascript object>

### Asi tendriamos una funcion para extraer los datos

In [23]:
base_url_UNPD = "https://population.un.org/dataportalapi/api/v1"
country = "4,8"  # set the country code
indicator_code = 60  # set the indicator code <-----ESTO SE CAMBIA
start_year = 1990  # set the start year
end_year = 2020  # set the end year
topick = (
    "mort"  # set the topic to change the parquet name file code <----- ESTO SE CAMBIA
)

# define the target URL
target = (
    base_url_UNPD
    + f"/data/indicators/{indicator_code}/locations/{country}/start/{start_year}/end/{end_year}"
)

response = requests.get(target)  # Call the API
j = response.json()  # Format response as JSON
df_UNPD = pd.json_normalize(j["data"])  # Read JSON data into dataframe

# As long as the response contains information in the 'nextPage' field, the loop will continue to download and append data
while j["nextPage"] is not None:
    response = requests.get(j["nextPage"])
    j = response.json()
    df_temp = pd.json_normalize(j["data"])
    df_UNPD = pd.concat([df_UNPD, df_temp], ignore_index=True)

df_UNPD

# Verifies that the number of records available from API call matches the length of the dataframe
# assert (
#     len(df_UNPD) == j["total"]
# ), "DataFrame observations do not match total number of records in response"

# df_UNPD.to_parquet(f"../datasets/df_UNPD_{topick}_{indicator_code}.parquet")

,locationId,location,iso3,iso2,locationTypeId,indicatorId,indicator,indicatorDisplayName,sourceId,source,revision,variantId,variant,variantShortName,variantLabel,timeId,timeLabel,timeMid,categoryId,category,estimateTypeId,estimateType,estimateMethodId,estimateMethod,sexId,sex,ageId,ageLabel,ageStart,ageEnd,ageMid,value
0,4,Afghanistan,AFG,AF,4,60,Total deaths by sex,Total deaths by sex,25,World Population Prospects,0,4,Median,Median,Median,41,1990,1990.5,0,Not applicable,1,Model-based Estimates,2,Interpolation,1,Male,188,Total,0,-1,0,110102
1,4,Afghanistan,AFG,AF,4,60,Total deaths by sex,Total deaths by sex,25,World Population Prospects,0,4,Median,Median,Median,41,1990,1990.5,0,Not applicable,1,Model-based Estimates,2,Interpolation,2,Female,188,Total,0,-1,0,93412
2,4,Afghanistan,AFG,AF,4,60,Total deaths by sex,Total deaths by sex,25,World Population Prospects,0,4,Median,Median,Median,41,1990,1990.5,0,Not applicable,1,Model-based Estimates,2,Interpolation,3,Both sexes,188,Total,0,-1,0,203514
3,4,Afghanistan,AFG,AF,4,60,Total deaths by sex,Total deaths by sex,25,World Population Prospects,0,4,Median,Median,Median,42,1991,1991.5,0,Not applicable,1,Model-based Estimates,2,Interpolation,1,Male,188,Total,0,-1,0,104274
4,4,Afghanistan,AFG,AF,4,60,Total deaths by sex,Total deaths by sex,25,World Population Prospects,0,4,Median,Median,Median,42,1991,1991.5,0,Not applicable,1,Model-based Estimates,2,Interpolation,2,Female,188,Total,0,-1,0,88257
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,8,Albania,ALB,AL,4,60,Total deaths by sex,Total deaths by sex,25,World Population Prospects,0,4,Median,Median,Median,70,2019,2019.5,0,Not applicable,1,Model-based Estimates,2,Interpolation,2,Female,188,Total,0,-1,0,10232
182,8,Albania,ALB,AL,4,60,Total deaths by sex,Total deaths by sex,25,World Population Prospects,0,4,Median,Median,Median,70,2019,2019.5,0,Not applicable,1,Model-based Estimates,2,Interpolation,3,Both sexes,188,Total,0,-1,0,24410
183,8,Albania,ALB,AL,4,60,Total deaths by sex,Total deaths by sex,25,World Population Prospects,0,4,Median,Median,Median,71,2020,2020.5,0,Not applicable,1,Model-based Estimates,2,Interpolation,1,Male,188,Total,0,-1,0,18095
184,8,Albania,ALB,AL,4,60,Total deaths by sex,Total deaths by sex,25,World Population Prospects,0,4,Median,Median,Median,71,2020,2020.5,0,Not applicable,1,Model-based Estimates,2,Interpolation,2,Female,188,Total,0,-1,0,12876


<IPython.core.display.Javascript object>

Lectura de un archivo parquet

In [26]:
read_data = pd.read_parquet("../data_lake/df_OMS_NCD_BMI_30A.parquet")
read_data

,Id,IndicatorCode,SpatialDim,TimeDimensionValue,Dim1Type,Dim1,NumericValue,Low,High
0,15433360.0,NCD_BMI_30A,SDF,1975,SEX,MLE,0.7,0.2,1.9
1,15433361.0,NCD_BMI_30A,SDF,1982,SEX,MLE,0.9,0.3,2.0
2,15433362.0,NCD_BMI_30A,SDF,1988,SEX,MLE,1.1,0.5,2.3
3,15433363.0,NCD_BMI_30A,SDF,1999,SEX,MLE,1.9,0.9,3.4
4,15433364.0,NCD_BMI_30A,SDF,2007,SEX,MLE,2.9,1.6,4.9
...,...,...,...,...,...,...,...,...,...
26040,NaN,None,None,None,None,None,NaN,NaN,NaN
26041,NaN,None,None,None,None,None,NaN,NaN,NaN
26042,NaN,None,None,None,None,None,NaN,NaN,NaN
26043,NaN,None,None,None,None,None,NaN,NaN,NaN


<IPython.core.display.Javascript object>

Revisamos una información básica de los dataframes

In [43]:
directory = "../data_lake/"
# Let's iterate each parquet file and get some basic information
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f) and f.endswith(".parquet"):
        print("========" + filename.rstrip(".parquet").upper() + "========", end="\n")
        df = pd.read_parquet(f)
        df.replace("", float("NaN"), inplace=True)
        print("====Basic info about the dataframe====", end="\n")
        print(df.info(), end="\n")
        print("====Sum the null in each column====", end="\n")
        print(df.isnull().sum(), end="\n")
        print("====Describe numeric columns====", end="\n")
        print(df.describe(), end="\n")
        print("====Get a sample of 5 rows====", end="\n")
        print(df.sample(5), end="\n")
        print(end="\n")

========DF_UNPD_MORT_22========
====Basic info about the dataframe====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21018 entries, 0 to 21017
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   location     21018 non-null  object 
 1   iso3         21018 non-null  object 
 2   timeLabel    21018 non-null  object 
 3   indicatorId  21018 non-null  int64  
 4   indicator    21018 non-null  object 
 5   sexId        21018 non-null  int64  
 6   sex          21018 non-null  object 
 7   value        21018 non-null  float64
dtypes: float64(1), int64(2), object(5)
memory usage: 1.3+ MB
None
====Sum the null in each column====
location       0
iso3           0
timeLabel      0
indicatorId    0
indicator      0
sexId          0
sex            0
value          0
dtype: int64
====Describe numeric columns====
       indicatorId         sexId         value
count      21018.0  21018.000000  21018.000000
mean          22.0    

====Basic info about the dataframe====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20959 entries, 0 to 20958
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Id                  20950 non-null  float64
 1   IndicatorCode       20950 non-null  object 
 2   SpatialDim          20950 non-null  object 
 3   TimeDimensionValue  20950 non-null  object 
 4   Dim1Type            20950 non-null  object 
 5   Dim1                20950 non-null  object 
 6   NumericValue        20900 non-null  float64
 7   Low                 20950 non-null  float64
 8   High                20950 non-null  float64
dtypes: float64(4), object(5)
memory usage: 1.4+ MB
None
====Sum the null in each column====
Id                     9
IndicatorCode          9
SpatialDim             9
TimeDimensionValue     9
Dim1Type               9
Dim1                   9
NumericValue          59
Low                    9
High                 

     countryiso3code  date         value  unit  obs_status  decimal  \
4188             HKG  2017  4.424254e+11   NaN         NaN        0   
5403             MRT  2011  1.520634e+10   NaN         NaN        0   
2466             BFA  2003  1.357655e+10   NaN         NaN        0   
3086             CUW  2003  3.102290e+09   NaN         NaN        0   
982              MNA  1999  1.871530e+12   NaN         NaN        0   

           indicator.id                                  indicator.value  \
4188  NY.GDP.MKTP.PP.CD  PIB, PPA ($ a precios internacionales actuales)   
5403  NY.GDP.MKTP.PP.CD  PIB, PPA ($ a precios internacionales actuales)   
2466  NY.GDP.MKTP.PP.CD  PIB, PPA ($ a precios internacionales actuales)   
3086  NY.GDP.MKTP.PP.CD  PIB, PPA ($ a precios internacionales actuales)   
982   NY.GDP.MKTP.PP.CD  PIB, PPA ($ a precios internacionales actuales)   

     country.id                                              country.value  
4188         HK                  Hong K

        locationId  locationTypeId  indicatorId  sourceId  revision  \
count  7006.000000          7006.0       7006.0    7006.0    7006.0   
mean    436.402655             4.0         53.0      25.0       0.0   
std     249.930958             0.0          0.0       0.0       0.0   
min       4.000000             4.0         53.0      25.0       0.0   
25%     222.000000             4.0         53.0      25.0       0.0   
50%     439.000000             4.0         53.0      25.0       0.0   
75%     646.000000             4.0         53.0      25.0       0.0   
max     894.000000             4.0         53.0      25.0       0.0   

       variantId      timeId  categoryId  estimateTypeId  estimateMethodId  \
count     7006.0  7006.00000      7006.0          7006.0            7006.0   
mean         4.0    56.00000         0.0             1.0               2.0   
std          0.0     8.94491         0.0             0.0               0.0   
min          4.0    41.00000         0.0        

====Basic info about the dataframe====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8246 entries, 0 to 8245
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   countryiso3code  8091 non-null   object 
 1   date             8246 non-null   object 
 2   value            5320 non-null   float64
 3   unit             0 non-null      float64
 4   obs_status       0 non-null      float64
 5   decimal          8246 non-null   int64  
 6   indicator.id     8246 non-null   object 
 7   indicator.value  8246 non-null   object 
 8   country.id       8246 non-null   object 
 9   country.value    8184 non-null   object 
dtypes: float64(3), int64(1), object(6)
memory usage: 644.3+ KB
None
====Sum the null in each column====
countryiso3code     155
date                  0
value              2926
unit               8246
obs_status         8246
decimal               0
indicator.id          0
indicator.value       0
count

...

========DF_UNPD_POP_50========
====Basic info about the dataframe====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7006 entries, 0 to 7005
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   locationId            7006 non-null   int64 
 1   location              7006 non-null   object
 2   iso3                  7006 non-null   object
 3   iso2                  7006 non-null   object
 4   locationTypeId        7006 non-null   int64 
 5   indicatorId           7006 non-null   int64 
 6   indicator             7006 non-null   object
 7   indicatorDisplayName  7006 non-null   object
 8   sourceId              7006 non-null   int64 
 9   source                7006 non-null   object
 10  revision              7006 non-null   int64 
 11  variantId             7006 non-null   int64 
 12  variant               7006 non-null   object
 13  variantShortName      7006 non-null   object
 14  variantLabel      

locationId              0
location                0
iso3                    0
iso2                    0
locationTypeId          0
indicatorId             0
indicator               0
indicatorDisplayName    0
sourceId                0
source                  0
revision                0
variantId               0
variant                 0
variantShortName        0
variantLabel            0
timeId                  0
timeLabel               0
timeMid                 0
categoryId              0
category                0
estimateTypeId          0
estimateType            0
estimateMethodId        0
estimateMethod          0
sexId                   0
sex                     0
ageId                   0
ageLabel                0
ageStart                0
ageEnd                  0
ageMid                  0
value                   0
dtype: int64
====Describe numeric columns====
        locationId  locationTypeId  indicatorId  sourceId  revision  \
count  7006.000000          7006.0       7006.0    

countryiso3code     155
date                  0
value              1102
dtype: int64
====Describe numeric columns====
             value
count  7144.000000
mean      4.218163
std       5.197455
min       0.000000
25%       0.673620
50%       2.460319
75%       6.208712
max      50.954034
====Get a sample of 5 rows====
     countryiso3code  date     value
1109             NaN  1996       NaN
768              LAC  1996  2.052857
830              LDC  1996  0.153393
1634             ASM  1998       NaN
7095             ESP  1993  5.540473

========DF_UNPD_MORT_24========
====Basic info about the dataframe====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21018 entries, 0 to 21017
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   location     21018 non-null  object 
 1   iso3         21018 non-null  object 
 2   timeLabel    21018 non-null  object 
 3   indicatorId  21018 non-null  int64  
 4   indicator    21018 no

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8246 entries, 0 to 8245
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   countryiso3code  8091 non-null   object 
 1   date             8246 non-null   object 
 2   value            7493 non-null   float64
 3   unit             0 non-null      float64
 4   obs_status       0 non-null      float64
 5   decimal          8246 non-null   int64  
 6   indicator.id     8246 non-null   object 
 7   indicator.value  8246 non-null   object 
 8   country.id       8246 non-null   object 
 9   country.value    8184 non-null   object 
dtypes: float64(3), int64(1), object(6)
memory usage: 644.3+ KB
None
====Sum the null in each column====
countryiso3code     155
date                  0
value               753
unit               8246
obs_status         8246
decimal               0
indicator.id          0
indicator.value       0
country.id            0
country.value       

<IPython.core.display.Javascript object>

In [28]:
df_high_income = df_copy[(df_copy["WorldBankIncomeGroup"] == "High-income countries")]
df_low_income = df_copy[(df_copy["WorldBankIncomeGroup"] == "Low-income countries")]
df_upper_middle_income = df_copy[
    (df_copy["WorldBankIncomeGroup"] == "Upper-middle-income countries")
]
df_lower_middle_income = df_copy[
    (df_copy["WorldBankIncomeGroup"] == "Lower-middle-income countries")
]

<IPython.core.display.Javascript object>

Revisamos una información básica de los dataframes

In [32]:
directory = "../data_lake/"
# Let's iterate each parquet file and get some basic information
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f) and f.endswith(".parquet"):
        print("========" + filename.rstrip(".parquet").upper() + "========", end="\n")
        df = pd.read_parquet(f)
        df.replace("", float("NaN"), inplace=True)
        print("====Basic info about the dataframe====", end="\n")
        print(df.info(), end="\n")
        print("====Sum the null in each column====", end="\n")
        print(df.isnull().sum(), end="\n")
        print("====Describe numeric columns====", end="\n")
        print(df.describe(), end="\n")
        print("====Get a sample of 5 rows====", end="\n")
        print(df.sample(5), end="\n")
        print(end="\n")

========DF_UNPD_MORT_22========
====Basic info about the dataframe====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21018 entries, 0 to 21017
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   location     21018 non-null  object 
 1   iso3         21018 non-null  object 
 2   timeLabel    21018 non-null  object 
 3   indicatorId  21018 non-null  int64  
 4   indicator    21018 non-null  object 
 5   sexId        21018 non-null  int64  
 6   sex          21018 non-null  object 
 7   value        21018 non-null  float64
dtypes: float64(1), int64(2), object(5)
memory usage: 1.3+ MB
None
====Sum the null in each column====
location       0
iso3           0
timeLabel      0
indicatorId    0
indicator      0
sexId          0
sex            0
value          0
dtype: int64
====Describe numeric columns====
       indicatorId         sexId         value
count      21018.0  21018.000000  21018.000000
mean          22.0    

countryiso3code    155
date                 0
value              102
dtype: int64
====Describe numeric columns====
             value
count  8144.000000
mean     55.759260
std      23.500073
min       5.416000
25%      35.841572
50%      54.861368
75%      74.797000
max     100.000000
====Get a sample of 5 rows====
     countryiso3code  date   value
2965             CRI  2000  59.052
7812             UKR  2020  69.608
2460             BFA  2009  24.079
2678             CAF  2008  38.506
5497             FSM  2010  22.298

========DF_OMS_NUTRITION_ANAEMIA_REPRODUCTIVEAGE_PREV========
====Basic info about the dataframe====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20959 entries, 0 to 20958
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Id                  20950 non-null  float64
 1   IndicatorCode       20950 non-null  object 
 2   SpatialDim          20950 non-null  object 
 3   TimeDimension

         locationId  locationTypeId  indicatorId  sourceId  revision  \
count  21018.000000         21018.0      21018.0   21018.0   21018.0   
mean     436.402655             4.0         49.0      25.0       0.0   
std      249.919066             0.0          0.0       0.0       0.0   
min        4.000000             4.0         49.0      25.0       0.0   
25%      222.000000             4.0         49.0      25.0       0.0   
50%      439.000000             4.0         49.0      25.0       0.0   
75%      646.000000             4.0         49.0      25.0       0.0   
max      894.000000             4.0         49.0      25.0       0.0   

       variantId        timeId  categoryId  estimateTypeId  estimateMethodId  \
count    21018.0  21018.000000     21018.0         21018.0           21018.0   
mean         4.0     56.000000         0.0             1.0               2.0   
std          0.0      8.944485         0.0             0.0               0.0   
min          4.0     41.000000 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21018 entries, 0 to 21017
Data columns (total 32 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   locationId            21018 non-null  int64 
 1   location              21018 non-null  object
 2   iso3                  21018 non-null  object
 3   iso2                  21018 non-null  object
 4   locationTypeId        21018 non-null  int64 
 5   indicatorId           21018 non-null  int64 
 6   indicator             21018 non-null  object
 7   indicatorDisplayName  21018 non-null  object
 8   sourceId              21018 non-null  int64 
 9   source                21018 non-null  object
 10  revision              21018 non-null  int64 
 11  variantId             21018 non-null  int64 
 12  variant               21018 non-null  object
 13  variantShortName      21018 non-null  object
 14  variantLabel          21018 non-null  object
 15  timeId                21018 non-null

                 Id  NumericValue           Low          High
count  2.095000e+04  20950.000000  20950.000000  20950.000000
mean   2.764218e+07     22.107828     15.510368     30.280635
std    1.087079e+06     19.668112     17.339564     21.885502
min    2.563744e+07      0.000000      0.000000      0.100000
25%    2.820283e+07      6.700000      3.000000     13.300000
50%    2.822378e+07     18.400000      9.400000     27.600000
75%    2.824473e+07     30.100000     21.000000     43.175000
max    2.828180e+07     88.400000     86.100000     91.600000
====Get a sample of 5 rows====
               Id                    IndicatorCode SpatialDim  \
740    25638182.0  NUTRITION_ANAEMIA_CHILDREN_PREV        COM   
10386  28223429.0  NUTRITION_ANAEMIA_CHILDREN_PREV        ITA   
9320   28219172.0  NUTRITION_ANAEMIA_CHILDREN_PREV        CHL   
6560   28208133.0  NUTRITION_ANAEMIA_CHILDREN_PREV        KAZ   
14003  28237888.0  NUTRITION_ANAEMIA_CHILDREN_PREV        GRD   

      TimeDimensionV

...

      locationId            location iso3 iso2  locationTypeId  indicatorId  \
532           64              Bhutan  BTN   BT               4           51   
3812         474          Martinique  MTQ   MQ               4           51   
2250         275  State of Palestine  PSE   PS               4           51   
1825         231            Ethiopia  ETH   ET               4           51   
1920         234       Faroe Islands  FRO   FO               4           51   

                      indicator                   indicatorDisplayName  \
532   Rate of population change  Crude rate of total population change   
3812  Rate of population change  Crude rate of total population change   
2250  Rate of population change  Crude rate of total population change   
1825  Rate of population change  Crude rate of total population change   
1920  Rate of population change  Crude rate of total population change   

      sourceId                      source  revision  variantId variant  \
532  

     countryiso3code  date      value
5859             NCL  2020  16.429001
3286             ECU  2020   6.110000
4664             KEN  2006   2.907000
75               ARB  2007  10.027224
5759             NAM  1996  22.768000

========PAISES========
====Basic info about the dataframe====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 178 entries, 0 to 177
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   name               178 non-null    object
 1   id                 178 non-null    object
 2   region.value       178 non-null    object
 3   incomeLevel.value  178 non-null    object
dtypes: object(4)
memory usage: 5.7+ KB
None
====Sum the null in each column====
name                 0
id                   0
region.value         0
incomeLevel.value    0
dtype: int64
====Describe numeric columns====
         name   id           region.value incomeLevel.value
count     178  178                    178   

====Basic info about the dataframe====
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21018 entries, 0 to 21017
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   location     21018 non-null  object 
 1   iso3         21018 non-null  object 
 2   timeLabel    21018 non-null  object 
 3   indicatorId  21018 non-null  int64  
 4   indicator    21018 non-null  object 
 5   sexId        21018 non-null  int64  
 6   sex          21018 non-null  object 
 7   value        21018 non-null  float64
dtypes: float64(1), int64(2), object(5)
memory usage: 1.3+ MB
None
====Sum the null in each column====
location       0
iso3           0
timeLabel      0
indicatorId    0
indicator      0
sexId          0
sex            0
value          0
dtype: int64
====Describe numeric columns====
       indicatorId         sexId         value
count      21018.0  21018.000000  21018.000000
mean          24.0      2.000000     43.389964
std    

             value  unit  obs_status  decimal
count  7493.000000   0.0         0.0   8246.0
mean     50.044809   NaN         NaN      1.0
std       2.539738   NaN         NaN      0.0
min      23.289054   NaN         NaN      1.0
25%      49.568379   NaN         NaN      1.0
50%      50.287478   NaN         NaN      1.0
75%      50.957705   NaN         NaN      1.0
max      54.564823   NaN         NaN      1.0
====Get a sample of 5 rows====
     countryiso3code  date      value  unit  obs_status  decimal  \
4849             KGZ  2007  50.484397   NaN         NaN        1   
7985             UZB  2002  50.296228   NaN         NaN        1   
3133             CZE  2018  50.808589   NaN         NaN        1   
6707             SEN  2009  51.289305   NaN         NaN        1   
1014             TMN  1998  49.472998   NaN         NaN        1   

           indicator.id                   indicator.value country.id  \
4849  SP.POP.TOTL.FE.ZS  Población, mujeres (% del total)         KG   
79

<IPython.core.display.Javascript object>

In [33]:
directory = "../data_lake/"
files = [f for f in os.listdir(directory) if f.endswith(".parquet")]

dfs = {file: pd.read_parquet(directory + file) for file in files}

count = []

for key, value in dfs.items():
    if key.startswith("df_TWB"):
        count.append(str(key.rstrip(".parquet").lstrip("df_TWB_")))
        print(value.head())

print(count)
# dfs["df_UNPD_mort_24.parquet"].loc[
#     :, dfs["df_UNPD_mort_24.parquet"].columns.str.fullmatch("indicator")
# ].mode()

  countryiso3code  date      value unit obs_status  decimal  \
0             AFE  2020  62.121491                        0   
1             AFE  2019  61.813185                        0   
2             AFE  2018  61.470354                        0   
3             AFE  2017  61.083445                        0   
4             AFE  2016  60.643170                        0   

        indicator.id                             indicator.value country.id  \
0  SP.DYN.LE00.MA.IN  Esperanza de vida al nacer, varones (años)         ZH   
1  SP.DYN.LE00.MA.IN  Esperanza de vida al nacer, varones (años)         ZH   
2  SP.DYN.LE00.MA.IN  Esperanza de vida al nacer, varones (años)         ZH   
3  SP.DYN.LE00.MA.IN  Esperanza de vida al nacer, varones (años)         ZH   
4  SP.DYN.LE00.MA.IN  Esperanza de vida al nacer, varones (años)         ZH   

  country.value  
0                
1                
2                
3                
4                
  countryiso3code  date      value
0  

<IPython.core.display.Javascript object>

In [34]:
UDPD = {
    "df_UNPD_mort_22": "tasa_mortalidad_infantil",
    "df_UNPD_pop_54": "densidad_población_por_kilómetro_cuadrado)",
    "df_UNPD_imigrt_65": "migración_neta_total",
    "df_UNPD_pop_49": "población_total_por_sexo",
    "df_UNPD_mort_60": "total_muertes_por_sexo",
    "df_UNPD_pop_53": "tasa_bruta_cambio_natural_población",
    "df_UNPD_imigrt_66": "tasa_bruta_migración_neta",
    "df_UNPD_pop_72": "proporción_sexos_población_total",
    "df_UNPD_fam_1": "prevalencia_anticonceptivos_porcentaje",
    "df_UNPD_pop_67": "mediana_edad_población",
    "df_UNPD_mort_59": "tasa_bruta_mortalidad_por_1000_habitantes",
    "df_UNPD_pop_51": "tasa_bruta_variación_total_población",
    "df_UNPD_pop_50": "cambio_de_la_población",
    "df_UNPD_pop_41": "población_femenina_edad_reproductiva_(15-49 años)",
    "df_UNPD_mort_24": "tasa_mortalidad_menores_cinco_años",
    "df_UNPD_pop_52": "cambio_natural_población",
    "df_UNPD_fert_19": "tasa_fertilidad",
    "df_UNPD_marstat_42": "estado_civil_casado_porcentaje",
}

<IPython.core.display.Javascript object>

In [35]:
WB = {
    "SP.DYN.LE00.IN": "esperanza_vida_total",
    "SP.DYN.LE00.FE.IN": "esperanza_vida_mujeres",
    "SP.DYN.LE00.MA.IN": "esperanza_vida_varones",
    "SI.POV.GINI": "índice_gini",
    "SE.XPD.TOTL.GD.ZS": "gasto_púb_educacion_pje",
    "SE.COM.DURS": "duración_educ_obligatoria",
    "NY.GDP.PCAP.CD": "pib_pc_usd_actuales",
    "NY.GDP.MKTP.PP.CD": "pib_ppa_prec_inter",
    "IQ.SCI.OVRL": "capacidad_estadística",
    "SP.POP.TOTL.FE.ZS": "población_mujeres_pje",
    "SP.POP.TOTL.MA.ZS": "población_hombres_pje",
    "NY.GDP.PCAP.PP.CD": "pib_pc_prec_inter",
    "AG.LND.FRST.ZS": "porcentaje_de_bosque",
    "EN.ATM.CO2E.PC": "emisiones_co2",
    "SH.XPD.CHEX.PC.CD": "inversion_salud_percapita",
    "SH.MED.BEDS.ZS": "camas_hospitales_c/1000personas",
    "SP.DYN.IMRT.IN": "mortalidad_infantil_c/1000nacimientos",
    "SH.H2O.BASW.ZS": "acceso_agua_potable(%)",
    "SH.STA.BASS.ZS": "acceso_servicios_sanitarios(%)",
    "SH.STA.SUIC.P5": "tasa_mortalidad_suicidio_c/100.000",
    "SL.UEM.TOTL.ZS": "tasa_desempleo",
    "SP.URB.TOTL.IN.ZS": "tasa_poblacion_urbana",
    "NY.GNP.PCAP.CD": "INB_percapita",
}

<IPython.core.display.Javascript object>

In [36]:
OMS = {
    "df_OMS_NUTRITION_ANAEMIA_CHILDREN_PREV": "tasa_anemia_niños(%)",
    "df_OMS_NUTRITION_ANAEMIA_REPRODUCTIVEAGE_PREV": "tasa_anemia_mujeres(%)",
    "df_OMS_M_Est_cig_curr": "tasa_consumo_cigarro(%)",
    "df_OMS_SA_0000001688": "tasa_consumo_alcohol(L)",
    "df_OMS_NCD_BMI_30A": "tasa_obesidad_pob(%)",
}

<IPython.core.display.Javascript object>

## Data Lake

![Alt text](https://linuxaria.com/wp-content/uploads/2016/07/gitlfs-768x403.png "a title")

In [45]:
df = pd.read_parquet("../data_lake/df_TWB_IQ.SCI.OVRL.parquet").sample(5)
df

,countryiso3code,date,value,unit,obs_status,decimal,indicator.id,indicator.value,country.id,country.value
1540,AFG,1999,NaN,,,1,IQ.SCI.OVRL,Nivel general de la capacidad estadística (escala 0 - 100),AF,Afganistán
351,ECA,2010,80.46772,,,1,IQ.SCI.OVRL,Nivel general de la capacidad estadística (escala 0 - 100),7E,Europa y Asia central (excluido altos ingresos)
7353,CHE,2014,NaN,,,1,IQ.SCI.OVRL,Nivel general de la capacidad estadística (escala 0 - 100),CH,Suiza
6879,SXM,1992,NaN,,,1,IQ.SCI.OVRL,Nivel general de la capacidad estadística (escala 0 - 100),SX,Sint Maarten (Dutch part)
6316,PER,1997,NaN,,,1,IQ.SCI.OVRL,Nivel general de la capacidad estadística (escala 0 - 100),PE,Perú


<IPython.core.display.Javascript object>

In [46]:
df = pd.read_parquet(
    "https://github.com/jorgeav527/life-expectancy/blob/main/data_lake/df_TWB_IQ.SCI.OVRL.parquet?raw=true"
).sample(5)
df

,countryiso3code,date,value,unit,obs_status,decimal,indicator.id,indicator.value,country.id,country.value
7531,TLS,1991,NaN,,,1,IQ.SCI.OVRL,Nivel general de la capacidad estadística (escala 0 - 100),TL,Timor-Leste
936,MEA,2014,NaN,,,1,IQ.SCI.OVRL,Nivel general de la capacidad estadística (escala 0 - 100),ZQ,Oriente Medio y Norte de África
4184,HND,1990,NaN,,,1,IQ.SCI.OVRL,Nivel general de la capacidad estadística (escala 0 - 100),HN,Honduras
4516,ITA,1999,NaN,,,1,IQ.SCI.OVRL,Nivel general de la capacidad estadística (escala 0 - 100),IT,Italia
1805,ABW,2013,NaN,,,1,IQ.SCI.OVRL,Nivel general de la capacidad estadística (escala 0 - 100),AW,Aruba


<IPython.core.display.Javascript object>

Las APIs usadas para la extraccion son la de las Naciones unidas, Banco Mundial y de la Organizacion de las Naciones Unidas, en cada una de ellas tenenemos una Documentacion especifica de cada enpoint.

Todas las API tienen como puntos principales la fuente de los datos, los topicos como Poblacion, Fertilidad, Mortalidad entre otras y una infinidad de inicadores los cuales nos ayudan a extraer la información de manera rapida y sencilla dando como parametros los paises, y rango en años.

Algunos problemas que surgieron en la extracción de los datos fueron, que la documentacion de algunas API esta desactualizadas y otra fue que algunas aveces el servidor se no daba respuesta.
 
Por tal motivo decidimos usar un git-lfs y usarlo como un data lake para no depender especificamente de la API y guardar los paises preseleccionados dentro del periodo de tiempo de 1990 al 2020 de cada uno de los indicadores en archivos en formato parquet que esta diseñado para admitir esquemas de compresión y codificación muy eficientes al ser tambien formato binario.

De esta manera podemos acceder a los datos de manera local y de manera remota.

